In [3]:
class Node():
    def __init__(self,state,parent,action):
        self.state = state                                 ## in this case ,Node class is keeping track of 
        self.parent = parent                                ## state ,parent , action
        self.action = action                         ### in this case , we are not keeping track of pathcost 
                                            ### as we will find it in the end from initial state to the goal

In [4]:
class QueueFrontier():                          ## class is a way to define objects
    def __init__(self):         ## i would like to create an object which is able to store my frontier data and 
        self.frontier = []         ##functions that will perform some operations
                                     ### here we have simple list
    def add(self, node):
        self.frontier.append(node)       ### it appends to the end of this list
        
    def contains_state(self, state):
        for node in self.frontier:
            if node.state == state:        ## it checks whether the frontier contains the given state
                return any(node)
                            
    def empty(self):
        return len(self.frontier) == 0 ## checks if the frontier is empty
    def remove(self):                                   ### operations will be same for stack
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0] ##[-1] ### as it is queue , it removes the first item in frontier
            self.frontier = self.frontier[1:]      ### we update the frontier with remaining items
            return node
    

In [5]:
class Maze():

    def __init__(self, filename):

        # Read file and set height and width of maze
        with open(filename) as f:
            contents = f.read()

        # Validate start and goal
        if contents.count("3") != 1:
            raise Exception("maze must have exactly one start point")
        if contents.count("G") != 1:
            raise Exception("maze must have exactly one goal")

        # Determine height and width of maze
        contents = contents.splitlines()
        self.height = len(contents)
        self.width = max(len(line) for line in contents)

        # Keep track of walls
        self.walls = []
        for i in range(self.height):
            row = []
            for j in range(self.width):
                try:
                    if contents[i][j] == "3":
                        self.start = (i, j)
                        row.append(False)
                    elif contents[i][j] == "G":
                        self.goal = (i, j)
                        row.append(False)
                    elif contents[i][j] == " ":
                        row.append(False)
                    else:
                        row.append(True)
                except IndexError:
                    row.append(False)
            self.walls.append(row)

        self.solution = None
        
    def print(self):
        solution = self.solution[1] if self.solution is not None else None
        print()
        for i, row in enumerate(self.walls):
            for j, col in enumerate(row):
                if col:
                    print("1", end="")
                elif (i, j) == self.start:
                    print("3", end="")
                elif (i, j) == self.goal:
                    print("G", end="")
                elif solution is not None and (i, j) in solution:
                    print("*", end="")
                else:
                    print(" ", end="")
            print()
        print()
    
    def neighbors(self, state):
        row, col = state
        candidates = [
            ("up", (row - 1, col)),
            ("down", (row + 1, col)),
            ("left", (row, col - 1)),
            ("right", (row, col + 1))
        ]

        result = []
        for action, (r, c) in candidates:
            if 0 <= r < self.height and 0 <= c < self.width and not self.walls[r][c]:
                result.append((action, (r, c)))
        return result
    
    
    def solve(self):
        """Finds a solution to maze, if one exists."""

        # Keep track of number of states explored
        self.num_explored = 0

        # Initialize frontier to just the starting position
        start = Node(state=self.start, parent=None, action=None)
        frontier = QueueFrontier()           ### AS BFS uses QUEUE as its data structure
        frontier.add(start)

        # Initialize an empty explored set
        self.explored = set()                ## to avoid repitition

        # Keep looping until solution found
        while True:

            # If nothing left in frontier, then no path
            if frontier.empty():
                raise Exception("no solution")

            # Choose a node from the frontier
            node = frontier.remove()
            self.num_explored += 1      ## #update number of states explored.

            # If node is the goal, then we have a solution
            if node.state == self.goal:
                actions = []         ## if i found out the goal...then i backtracktofind whch actions i took
                cells = []
                while node.parent is not None:  ## getting back to the parent node unless we find initial node
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()                      ## we reverse so that we get the right sequence
                cells.reverse()
                self.solution = (actions, cells)
                return self.solution

            # Mark node as explored
            self.explored.add(node.state)
                
            # Add neighbors to frontier
            for action, state in self.neighbors(node.state):
                if not frontier.contains_state(state) and state not in self.explored:
                    child = Node(state=state, parent=node, action=action)
                    frontier.add(child)
                    
        
    

In [6]:
m = Maze("maze.txt")
print("Maze:")
m.print()

m.solve()
print("States Explored:", m.num_explored)
print("path cost is :", m.num_explored) ### one jump cost is 1....
print("Solution:")
m.print()

Maze:

3   1
111G1
11111
11111
11111

States Explored: 5
path cost is : 5
Solution:

3***1
111G1
11111
11111
11111

